In [1]:
import torch

In [2]:
torch.__version__

'1.5.0'

In [3]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm.autonotebook import tqdm

import torch
from torch.utils.data import DataLoader

In [4]:
root_dir1 = "/home/trung/MOTChallenge2020"

In [5]:
sys.path.append(os.path.join(root_dir1, 'src'))

In [6]:
from tracker.data_track import MOT16Sequences
from tracker.data_obj_detect import MOT16ObjDetect
from tracker.object_detector import FRCNN_FPN
from tracker.tracker import Tracker
from tracker.utils import (plot_sequence, evaluate_mot_accums, get_mot_accum,
                           evaluate_obj_detect, obj_detect_transforms, collate_fn)
import motmetrics as mm
mm.lap.default_solver = 'lap'

In [7]:
seq_name = 'MOT16-02'
root_dir = '/home/trung/MOTChallenge2020_data'
data_dir = os.path.join(root_dir, 'data/MOT16')
sequences = MOT16Sequences(seq_name, data_dir)


In [8]:
obj_detect_model_file = os.path.join(root_dir, 'models/faster_rcnn_fpn.model')
obj_detect_nms_thresh = 0.3

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
# object detector
obj_detect = FRCNN_FPN(num_classes=2, nms_thresh=obj_detect_nms_thresh)
obj_detect_state_dict = torch.load(obj_detect_model_file,
                                   map_location=lambda storage, loc: storage)
obj_detect.load_state_dict(obj_detect_state_dict)
obj_detect.eval()
obj_detect.to(device)

cuda


FRCNN_FPN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequenti

In [10]:
from torchvision.ops import roi_align
for seq in sequences:# Each seq is a MOT16Sequence object

    print(f"Tracking: {seq}")
    inc = 0
    list_boxes = []
    list_img = []

    data_loader = DataLoader(seq, batch_size=6, shuffle=False, collate_fn = collate_fn) #NOTE: shuffle  = False. This is order sequence !!!!
    print("Done loading data")
    for batch_frame in tqdm(data_loader):
        boxes, scores = obj_detect.detect(torch.unsqueeze(batch_frame['img'][0],0))
        list_boxes.append(boxes)
        list_img.append(batch_frame['img'][0])
        inc = inc+1
        if inc==15: 
            break
    batch_images = torch.stack(list_img)
    detections = roi_align(batch_images, list_boxes, (128,64))       
    

Tracking: MOT16-02
Done loading data


In [ ]:
print(batch_images.size())
print(detections.size())

In [ ]:
from tracker.feature_encoder import NodeEncoder

In [ ]:
node_encoder = NodeEncoder()
result = node_encoder(detections)

In [ ]:
print(batch_images.size())
print(detections.size())
print(result.size())

In [ ]:
from torchvision.ops import roi_align
for seq in sequences:# Each seq is a MOT16Sequence object

    print(f"Tracking: {seq}")
    list_boxes = []
    list_img = []
    inc = 0
    

    data_loader = DataLoader(seq, batch_size=1, shuffle=False) #NOTE: shuffle  = False. This is order sequence !!!!
    print("Done loading data")
    for frame in tqdm(data_loader):
        boxes, scores = obj_detect.detect(frame['img'])
        list_boxes.append(boxes)
        list_img.append(torch.squeeze(frame['img']))
        inc = inc+1
        if inc==5: 
            break
    batch_images = torch.stack(list_img)
    detections = roi_align(batch_images, list_boxes, (128,64))
 
        
        